<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span><ul class="toc-item"><li><span><a href="#Будем-использовать-шифр-Хилла." data-toc-modified-id="Будем-использовать-шифр-Хилла.-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Будем использовать шифр Хилла.</a></span></li></ul></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span><ul class="toc-item"><li><span><a href="#Признаки-умножают-на-обратимую-матрицу.-Изменится-ли-качество-линейной-регрессии?" data-toc-modified-id="Признаки-умножают-на-обратимую-матрицу.-Изменится-ли-качество-линейной-регрессии?-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?</a></span></li></ul></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Вывод</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression

from sklearn.metrics import r2_score, mean_squared_error

SEED = 41

In [2]:
try:
    df = pd.read_csv('../datasets/insurance.csv')
except:
    df = pd.read_csv('/datasets/insurance.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [5]:
df.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Умножим матрицу признаков на обратимую матрицу P

$$
a' = (XP)w'
$$

Тогда формула обучения:

$$
w' = ((XP)^T XP)^{-1} (XP)^T y = (P^T X^T X P)^{-1} P^T X^T y = P^{-1} (X^TX)^{-1} (P^T)^{-1} P^T X^T y 
$$
- $(P^T)^{-1}P^T$ - сократятся. 

$$
w' = P^{-1} (X^TX)^{-1} X^Ty
$$
тогда 
$$
a' = (XP)P^{-1} (X^TX)^{-1} X^Ty = XPP^{-1} (X^TX)^{-1} X^Ty
$$
- $P P^{-1}$ - сократятся
$$
a' = X(X^TX)^{-1}X^Ty
$$

**Ответ:** Качество не изменится

**Обоснование:** Мы сгенерировали матрицу из случайного распределения. При умножении признаки изменились, однако линейная взаимосвязь не исчезла, поэтому на качество модели это не повлияло. 


## Алгоритм преобразования

###  Будем использовать шифр Хилла.
- Создадим матрицу-ключ размера n на n, где n - ширина нашей матрицы признаков X
- Проверим матрицу-ключ на обратимость.
- Умножим матрицу признаков на матрицу-ключ 
- Проверим качество метрики R2 на незашифрованных данных
- Проверим качество метрики R2 после умножения 
- Сравним получившиеся результаты.

**Итог** - данные расшифровываются с помощью обратной матрицы-ключа.

## Проверка алгоритма

In [6]:
# Функция для обучения и оценки.

def fit_and_evaluate(features_train, target_train, features_test, target_test, 
                     multiply:bool=False, matrix_key=None, verbose:bool = False):
    
    model = LinearRegression() # Объявляем модель
    
    if multiply: # флаг, он позволит сравить результаты без умножения и с умножением
        try:
            np.linalg.inv(matrix_key)
        except:
            return  print("Переданная в функцию матрица необратима ! Передайте в функцию обратимую матрицу")
        features_train = features_train.values @ matrix_key # Умножаем признаки тренировочной выборки на созданную матрицу
        features_test = features_test.values @ matrix_key # Умножаем признаки тестовой выборки 
    model.fit(features_train, target_train) # Обучаем модель
    predictions = model.predict(features_test) # Делаем предсказания
    
    r2 = r2_score(target_test, predictions) # считаем  r2
    mse = mean_squared_error(target_test, predictions) # считаем MSE
    if verbose:
        print('-'*80)
        print(f'r2 score: {r2:.5}')
        print(f'MSE: {mse:.5}')
        print('-'*80)
        
    return features_train, features_test, predictions

### Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?

In [7]:
# Разбиение данных
X_train, X_test, y_train, y_test = train_test_split(
    df.drop('Страховые выплаты', axis=1),
    df['Страховые выплаты'],
    test_size=.25,
    random_state=SEED   
)

In [8]:
# Сначала смотрим на базовые метрики нашей выборки 
train, test, predictions = fit_and_evaluate(X_train, y_train, X_test, y_test, verbose=1)

--------------------------------------------------------------------------------
r2 score: 0.42603
MSE: 0.11391
--------------------------------------------------------------------------------


In [9]:
# Теперь умножаем на матрицу и сравниваем результаты 
matrix_key = np.random.normal(size=(4,4))
train_encrypted, test_encrypted, predictions = fit_and_evaluate(X_train, y_train, X_test, y_test, verbose=True,
                multiply=True, matrix_key=matrix_key)


--------------------------------------------------------------------------------
r2 score: 0.42603
MSE: 0.11391
--------------------------------------------------------------------------------


In [10]:
# Убедимся, что алгоритм дешифровки работает корректно. 
display(pd.DataFrame(train_encrypted, columns = X_train.columns).head(5))
pd.DataFrame(train_encrypted @ np.linalg.inv(matrix_key), 
             columns = X_train.columns).astype(int).head(5)

,Пол,Возраст,Зарплата,Члены семьи
0,56657.247811,-46634.311210,64199.215341,-2436.465841
1,35613.509238,-29308.500921,40299.807842,-1510.748238
2,51534.797770,-42417.413413,58387.045037,-2213.279713
3,48900.144974,-40249.955877,55413.689535,-2104.763096
4,76183.667768,-62709.152150,86352.093081,-3287.857052


,Пол,Возраст,Зарплата,Члены семьи
0,0,30,40900,0
1,0,34,25699,0
2,1,28,37199,0
3,0,24,35300,1
4,0,31,55000,2


## Вывод

- На вход нам подаются данные клиентов страховой компании. Данные не имеют существенных аномалий и выбросов, не требуют предобработки
- Мы доказали математически, что качество линейной регрессии не меняется при умножении признаков на обратимую матрицу. 
- Далее мы доказали это на примере. 
- Используя шифр Хилла мы закодировали данные с помощью матрицы-ключа. 
- В последнем пункте мы расшифровали закодированные данные с помощью обратной матрицы-ключа, убедились в корректной работе алгоритма. 
